In [58]:
import os
import cv2
import numpy as np
import tqdm
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [24]:
CATEGORIES = ['False', 'True']
IMG_SIZE = 116

In [73]:
training_data = []

def creating_data():
    for category in CATEGORIES:
        path = category
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            # to gray scale
            img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
            training_data.append([img_array, class_num])

creating_data()

In [75]:
import random
random.shuffle(training_data)

In [84]:
X = []
y = []

In [85]:
for features, labels in training_data:
    X.append(features)
    y.append(labels)

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
#scaling
X = X/255

y = np.array(y)

In [86]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=.3, random_state=27)

In [102]:
#model initialization
model = keras.models.Sequential([
    keras.layers.Conv2D(filters=6, kernel_size=3, padding='same', activation='relu'), # -> 6, 116, 116
    keras.layers.MaxPool2D((2, 2)), # -> 6, 58, 58
    
    keras.layers.Conv2D(filters=12, kernel_size=3, padding='same', activation='relu'), # -> 12, 58, 58
    keras.layers.MaxPool2D((2, 2)), # -> 12, 29, 29

    keras.layers.Flatten(), # -> 12*29*29
    
    keras.layers.Dense(64, activation='relu'), # -> 64
    keras.layers.Dense(1, activation='sigmoid')

])

In [103]:
#optimizer
optimizer = keras.optimizers.Adam(learning_rate = 0.001 , beta_1 = 0.9 , beta_2 = 0.99)
model.compile( optimizer=optimizer,
              loss='binary_crossentropy',  metrics=[keras.metrics.Precision(), keras.metrics.Recall()])

In [104]:
#fit
model.fit(X_train, y_train,
                batch_size=250,
                epochs=3,
                validation_data=(X_valid, y_valid))

Epoch 1/3
106/106 [==============================] - 44s 394ms/step - loss: 0.0643 - precision_3: 0.9900 - recall_3: 0.9190 - val_loss: 0.0071 - val_precision_3: 0.9978 - val_recall_3: 0.9951
Epoch 2/3
106/106 [==============================] - 42s 400ms/step - loss: 0.0041 - precision_3: 0.9985 - recall_3: 0.9972 - val_loss: 0.0027 - val_precision_3: 0.9985 - val_recall_3: 0.9985
Epoch 3/3
106/106 [==============================] - 42s 398ms/step - loss: 0.0023 - precision_3: 0.9988 - recall_3: 0.9992 - val_loss: 0.0016 - val_precision_3: 0.9985 - val_recall_3: 0.9981
